# Prédiction de la Performance énergétique des communes en France

###  Cartographie des niveaux de Diagnostic de Performance énergétique et des niveaux de vie Par commune

Le but ultime de cette section est de construire des cartes interactives représentant les niveau de vie et les DPE majoritaires dans les arrondissements

In [1]:
import pandas as pd

KeyboardInterrupt: 

In [ ]:
data = pd.read_csv("~/work/data_science_project/Filosofi_dpe_IDF_com.csv")
#data.head()

In [ ]:
# On télécharge les coordonnées géographiques des communes

In [ ]:
!pip install --upgrade botocore==1.23.26  
!pip install --upgrade urllib3==1.22.0 
!pip install py7zr 
!pip install s3fs 
!git clone https://github.com/InseeFrLab/cartogether.git
%cd ./cartogether
!pip install -r requirements.txt
!pip install .

In [ ]:
import cartiflette.s3

communes_IDF = cartiflette.s3.download_vectorfile_url_all(
    values = ["75","91","92","93","94","95","77","78"],
    level="COMMUNE",
    vectorfile_format="geojson",
    decoupage="departement",
    year=2022)

#communes_IDF.head()

In [ ]:
# On se rassure que les codes communes sont uniques par liqne
communes_IDF["INSEE_COM"].value_counts().sum()

In [ ]:
# De même pour les codes communes des données de DPE
data['CODGEO'].value_counts().sum()

In [ ]:
#communes_IDF.info()

In [ ]:
#data.columns

In [ ]:
# Création d'une colonne pour la proportion des ménages énergivores
data['Energ'] = data['classe_consommation_energie_E'] + data['classe_consommation_energie_F'] + data['classe_consommation_energie_G']

In [ ]:
Cols = ['CODGEO', 'LIBGEO', 'NBMENFISC16', 'NBPERSMENFISC16', 'MED16','surface_thermique_lot_median', 
        'age_logement_median','consommation_energie_median', 'nombre_dpe', 'Energ', 'classe_consommation_energie_A',
       'classe_consommation_energie_B','classe_consommation_energie_C','classe_consommation_energie_D','classe_consommation_energie_E',
       'classe_consommation_energie_F','classe_consommation_energie_G']
Filo_dpe_cart = data[Cols]
#Filo_dpe_cart.info()

In [ ]:
Filo_dpe_cart.rename(columns={'CODGEO':'INSEE_COM'}, inplace=True)

In [ ]:
# Filo_dpe_cart.head()

In [ ]:
#Filo_dpe_cart.shape

In [ ]:
#communes_IDF.shape

In [ ]:
# Fonction qui calcule la classe d'énergie majoritaire dans une commune
def maj(x):
    dic = 'A B C D E F G'.split()
    list = [x.classe_consommation_energie_A, x.classe_consommation_energie_B, x.classe_consommation_energie_C, 
            x.classe_consommation_energie_D, x.classe_consommation_energie_E, x.classe_consommation_energie_F,
            x.classe_consommation_energie_G]
    indice = [i for i, item in enumerate(list) if item == max(list)]
    if len(indice) != 0:
        return([ dic[j] for j in indice][len(indice)-1])
    else:
        return 'N'

In [ ]:
#maj(Filo_dpe_cart.iloc[0,:])

In [ ]:
# Création d'une colonne avec ce maximum
Filo_dpe_cart['CE_Maj'] = Filo_dpe_cart.apply(lambda z: maj(z),  axis=1)

In [ ]:
#Filo_dpe_cart.head()

In [ ]:
#Filo_dpe_cart.groupby('CE_Maj')['INSEE_COM'].count()

In [ ]:
communes_IDF = communes_IDF[['NOM','INSEE_COM','geometry']]
#communes_IDF.head()

In [ ]:
# Jointure des communes, et de leurs informations
communes_IDF['INSEE_COM'] = communes_IDF['INSEE_COM'].apply(lambda n: int(n))
data_used = communes_IDF.merge(Filo_dpe_cart, on = 'INSEE_COM', how='right')

#data_used.columns
#data_used.info()

In [ ]:
data_used = data_used.dropna(subset=["geometry"])
#data_used.info()

In [ ]:
# Vue sur le revenu médian
ax = data_used.plot(figsize=(10,10), column = 'MED16', legend = True)
ax.set_axis_off()

In [ ]:
data_used.info()